# Tracing and Testing

This notebook contains the experiment to investigate the question **Can we control the outbreak using only testing via contact tracing, i.e. testing people that had overlap with positively tested, and then subsequently isolating positively tested?**.


* In Section 1, we define all the simulation parameters.
* In Section 2, we run all the simulations needed for the experiment.
* In Section 3, we plot the results.

To just regenerate the figures from a summary file containing all the simulation objects of the experiment, you can skip Section 2, and only run the cells in Sections 1 & 3. 

---

## 1. Define the experiment parameters

#### Import libs

In [1]:
%load_ext autoreload
%autoreload 2
import sys
if '..' not in sys.path:
    sys.path.append('..')

In [2]:
import numpy as np
import pandas as pd
import pickle
import multiprocessing

In [3]:
from lib.measures import *
from lib.experiment import run_experiment, save_summary, load_summary
from lib.calibrationSettings import calibration_lockdown_dates
from lib.calibrationFunctions import get_calibrated_params

## 1. General settings  

#### Set the random seed for reproducibility

In [4]:
# Choose random seed
c = 0
np.random.seed(c)
TO_HOURS = 24.0

# Define prefix string used to save plots
expstr = 'tracing-testing'
multi_beta_calibration = False

#### Set the number of roll-outs to simulate

In [5]:
random_repeats = 2 # Set to at least 40 to obtain stable results

#### Set the time to simulate

In [6]:
end_date = '2020-07-31'

#### Define locations, mobility settings used for simulation, and experiment parameters

In [7]:
locs = {
    'GER': {
        'TU': 'lib/mobility/Tubingen_settings_10.pk',
        'KL': 'lib/mobility/Kaiserslautern_settings_10.pk',
        'RH': 'lib/mobility/Ruedesheim_settings_10.pk', 
        'TR': 'lib/mobility/Tirschenreuth_settings_10.pk', 
    },
    'CH': {
        'VD': 'lib/mobility/Lausanne_settings_10.pk', 
        'LU': 'lib/mobility/Lucerne_settings_5.pk', 
        'TI': 'lib/mobility/Locarno_settings_2.pk', 
        'JU': 'lib/mobility/Jura_settings_10.pk',
    }
}

seed_summary_path = None

# experiment parameters
test_smart_delta = 3 * TO_HOURS # time window considered for inspecting contacts
test_smart_action = 'test' # isolate traced individuals
test_targets = 'isym'
contacts_tested = [10, 25] # how many contacts are tested from the `test_smart_delta` window


## 2. Run the simulations

Use settings as above and simulate in the future with additional measures

**WARNING: the following cells might take a long time to run depending of the parameters defined above!**

In [ ]:
for country, areas in locs.items():
    for area, mob_settings in areas.items():
        s = dict()
        
        # start simulation when lockdown ends
        start_date = calibration_lockdown_dates[country]['end']
        
        # load social distancing parameter
        p_stay_home = get_calibrated_params(country=country, area=area, multi_beta_calibration=multi_beta_calibration)['p_stay_home']
        
        # baseline
        baseline = run_experiment(
            country=country, 
            area=area, 
            mob_settings=mob_settings,
            start_date=start_date, 
            end_date=end_date, 
            measure_list=[], 
            random_repeats=random_repeats,
            test_update=None, 
            seed_summary_path=seed_summary_path,
            multi_beta_calibration=multi_beta_calibration)
        
        s['baseline'] = baseline
        
        print('Baseline done.', flush=True)

        # experiment
        for contacts in contacts_tested:
            for policy in policies:

                # no additional measures
                m = []

                # update testing params using this function
                def test_update(d):
                    d['test_smart_delta'] = test_smart_delta
                    d['test_smart_action'] = test_smart_action
                    d['test_targets'] = test_targets
                    d['smart_tracing'] = policy
                    d['test_smart_num_contacts'] = contacts
                    return d

                # run
                res = run_experiment(
                    country=country, 
                    area=area, 
                    mob_settings=mob_settings,
                    start_date=start_date, 
                    end_date=end_date, 
                    measure_list=m, 
                    random_repeats=random_repeats,
                    test_update=test_update, 
                    seed_summary_path=seed_summary_path,
                    multi_beta_calibration=multi_beta_calibration)

                s[(contacts, policy)] = res

                print(country, area, contacts, policy, ' done.', flush=True)

        # save results
        save_summary(s, f'state-{expstr}--{country}-{area}.pk')

## 3. Plot the results

Import libs

In [ ]:
from lib.plot import Plotter

Create figures from summaries

In [ ]:
for country, areas in locs.items():
    for area in areas:
        s = load_summary(f'state-{expstr}--{country}-{area}.pk')
        
        for contacts in contacts_isolated:
            plotter = Plotter()
            titles = ['No tracing', 'Basic tracing', 'Advanced tracing']
            summaries = [
                s['baseline'],
                s[(contacts, 'basic')],
                s[(contacts, 'advanced')],
            ]

            fn = f'{expstr}--{country}-{area}--{contacts}'
            print(fn)
            plotter.compare_total_infections(
                summaries, 
                titles=titles, 
                filename=fn, 
                start_date=calibration_lockdown_dates[country]['end'],
                figsize=(6,4), acc=500, 
                ymax=500, errorevery=14)
